In [ ]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'h2_dashboard.settings')
django.setup()

In [2]:
from django.apps import apps

def get_benchmark_data():
    # List of benchmark models you have, e.g., ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_models = ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_data = {}

    for model_name in benchmark_models:
        model = apps.get_model('h2_prod', model_name)  # Adjust 'h2_prod' to your app name if different
        benchmarks = model.objects.all()

        for benchmark in benchmarks:
            key = f"{model_name}-{benchmark.year}"
            if key not in benchmark_data:
                benchmark_data[key] = []
            benchmark_data[key].append(benchmark.value)

    return benchmark_data


In [5]:
def current_total_co2_emissions(total_h2_production):
    # Percentage of hydrogen production from each source
    percentage_gas = 0.48
    percentage_oil = 0.3
    percentage_coal = 0.18
    percentage_electro = 0.04

    # CO2 emissions (kg) from each source
    co2_per_kg_gas = 12  # kg CO2e per kg H2
    co2_per_kg_oil = 18  # kg CO2e per kg H2
    co2_per_kg_coal = 24  # kg CO2e per kg H2
    co2_per_kg_electro = 22  # kg CO2e per kg H2

    # Convert total hydrogen production from megatonnes to kilograms (1 megatonne = 1,000,000,000 kg)
    total_h2_prod_kg = total_h2_production * 1e9

    # Calculating the CO2 emissions from each source
    co2_emissions_gas = total_h2_prod_kg * percentage_gas * co2_per_kg_gas
    co2_emissions_oil = total_h2_prod_kg * percentage_oil * co2_per_kg_oil
    co2_emissions_coal = total_h2_prod_kg * percentage_coal * co2_per_kg_coal
    co2_emissions_electro = total_h2_prod_kg * percentage_electro * co2_per_kg_electro

    # Summing up the CO2 emissions from all sources to get the total CO2 emissions from hydrogen production
    previous_co2_emission = co2_emissions_gas + co2_emissions_oil + co2_emissions_coal + co2_emissions_electro
    return previous_co2_emission * 1e-9  # Conversion to Mega tonnes

print(round(current_total_co2_emissions(120)))

1963


Customize Your Templates:

Begin integrating the AdminLTE template into your Django views. Replace the HTML structure in your Django templates with the structure from AdminLTE's HTML files, making sure to keep Django template tags and context variables intac

unit font size
move reset button to top left
remove calculate button
slider to stay in place, only return to default place when reset button is clicked

In [5]:
electrolyzer_rating = 20 # MW 
kwh_per_kg_h2 = 33.3 # kWh 
default_total_h2_production = 120  # megatonnes per year
electrolyzer_efficiency = 70 # percent
total_h2_production = 90 # megatonnes per year


electricity_per_kg_h2 = kwh_per_kg_h2 / (electrolyzer_efficiency / 100)
total_electricity_requirement = electricity_per_kg_h2 * total_h2_production * 1e9  # Conversion to kg
electrolyzer_consumption = electrolyzer_rating * 8760 # per year running 24/7
required_electrolyzer_units = (total_electricity_requirement*1e-3)/(electrolyzer_rating * 8760) 

print(round(total_electricity_requirement))
print(round(electrolyzer_consumption))
print(required_electrolyzer_units)

4281428571429
175200
24437.37769080235


In [5]:
post_data = {
    'csrfmiddlewaretoken': 'some_token',
    'hiddenH2Production': '152',
    'hiddenElectrolyzerEff': '70',
    'hiddenRenewablePct': '30',
    'hiddenCO2Emission': '0.47'
}



<class 'str'>


In [6]:
def convert_str_float(post_data):



    # Define the keys you expect to convert to float
    keys_to_convert = ['hiddenH2Production', 'hiddenElectrolyzerEff', 'hiddenRenewablePct', 'hiddenCO2Emission']

    # Create a dictionary to hold the converted values
    converted_values = {}

    # Iterate over the keys and convert the values
    for key in keys_to_convert:
        try:
            converted_values[key] = float(post_data.get(key, 0))
        except ValueError:
            # Handle the case where conversion fails
            print(f"Value conversion failed for key: {key}")
            converted_values[key] = 0  # or handle this differently as per your need

    # Logging the converted values and their types
    for key, value in converted_values.items():
        print(f"{key}: {value}, Type: {type(value)}")
    
    return converted_values

data = convert_str_float(post_data)
print(data)

In [8]:
def post_floater(post_data):
    for key, value in post_data.items():
        if key != 'csrfmiddlewaretoken':
            try:
                post_data[key] = float(value)
            except ValueError:
                # Handle the case where conversion fails
                print(f"Value conversion failed for key: {key}")
                post_data[key] = 0  # or handle this differently as per your need

    # Logging the converted values and their types
    for key, value in post_data.items():
        print(f"{key}: {value}, Type: {type(value)}")
    
    return post_data

print(post_floater(post_data))

{'csrfmiddlewaretoken': 'some_token', 'hiddenH2Production': 152.0, 'hiddenElectrolyzerEff': 70.0, 'hiddenRenewablePct': 30.0, 'hiddenCO2Emission': 0.47}


In [ ]:
def hydrogen_production_view(request):
    #  debugging
    print("View function called")
    print("Request method:", request.method)

    # context initialization
    context = {'form': HydrogenProductionForm()}

    # initial values
    initial_values = {
        'initial_h2_production': 90,  # MegaTonnes
        'initial_electrolyzer_efficiency': 70,  # Percent
        'initial_renewable_percentage': 30,  # Percent
        'initial_co2_emission_per_kwh_fossil': 0.47,  # kg
        'initial_total_electricity_requirement': 4281,  # TWh
        'initial_required_electrolyzer_units': 24437,  # Units
        'initial_total_co2_emissions': 1409  # MegaTonnes
    }

    context.update(initial_values)

    # if request.method == 'POST':for reset

        print("Reset button clicked")
        # Reset the form with initial values
        context = {'form': HydrogenProductionForm(initial=initial_values)}
        return render(request, 'h2_prod/hydrogen_production.html', context)


    # if request.method == 'POST':for calculation update
 
        print("POST request received")
        print("Post Data:", request.POST)

        # convert the POST strings data to float
        post_data = request.POST.copy()
        post_data = post_floater(post_data)



        # Production & electrolizer Efficiency
        total_h2_production = post_data.get('hiddenH2Production', None)
        electrolyzer_efficiency = post_data.get('hiddenElectrolyzerEff', None)
        # Grid renewable percentage & total CO2e emission
        co2_emission_per_kwh_fossil = post_data.get('hiddenCO2Emission', None)
        renewable_percentage = post_data.get('hiddenRenewablePct', None)

        # state of CO2 emissions
        current_state_co2_emission = current_total_co2_emissions(total_h2_production)

        # calculations for the new state of H2 production, 
        # how much CO2 emission is reduced, and how much electricity is required 
        total_electricity_requirement, total_co2_emissions, co2_emissions_reduction, total_h2_production, required_electrolyzer_units = calculate_outputs(
        total_h2_production, electrolyzer_efficiency, renewable_percentage, 
        co2_emission_per_kwh_fossil, current_state_co2_emission, electrolyzer_rating)

        # round the total_electricity_requirement and total_co2_emissions
        rounded_total_electricity_requirement = round(total_electricity_requirement)
        rounded_total_co2_emissions = round(total_co2_emissions)
        rounded_total_h2_production = round(total_h2_production)
        rounded_required_electrolyzer_units = round(required_electrolyzer_units)


        # Create the bar chart

        # step 1: fetch benchmark data
        benchmark_data = get_benchmark_data()

        # step 2: separate the benchmark data into two lists
        
        # electricity_benchmark_data
        electricity_benchmark_data = sorted(
            [(int(key.split('-')[1]), benchmark_value, benchmark_name)
            for key, values in benchmark_data.items() if 'ElectricityProductionBenchmark' in key
            for benchmark_value, benchmark_name in values],
            key=lambda x: x[0]
        )

        # co2_benchmark_data
        co2_benchmark_data = sorted(
            [(int(key.split('-')[1]), benchmark_value, benchmark_name)
            for key, values in benchmark_data.items() if 'CO2EmissionsBenchmark' in key
            for benchmark_value, benchmark_name in values],
            key=lambda x: x[0]
        )

        # benchmark_colors
        benchmark_colors = ['#d53e4f', '#f46d43', '#fdae61']
        reversed_colors = benchmark_colors[::-1]

        # step 3: Prepare data for the chart
        # Electricity data
        electricity_chart_data = [
            # Add the benchmark data to the chart
            *[
            go.Bar(
                x = [f'{name}'],
                y = [value],
                name = f'Benchmark {year}',
                marker = dict(color=reversed_colors[i])
            )
            for i, (year, value, name) in enumerate(electricity_benchmark_data[:3])
            ],

            # Add the calculated value to the chart
            go.Bar(
                x = ['Your Calculation'],
                y = [rounded_total_electricity_requirement],
                name = 'Your Calculation',
                marker = dict(color='#3288bd')
            )
        ]

        # CO2 data
        co2_chart_data = [
            # Add the benchmark data to the chart
            *[
            go.Bar(
                x = [f'{name}'],
                y = [value],
                name = f'Benchmark {year}',
                marker = dict(color=reversed_colors[i])
            )
            for i, (year, value, name) in enumerate(co2_benchmark_data[:3])
            ],

            # Add the current state of CO2 emissions to produce hydrogen
            go.Bar(
                x = ['Current State of CO2 Emissions to produce H2'],
                y = [round(current_total_co2_emissions(120))],
                name = 'Your Calculation',
                marker = dict(color='#9e0142')
            ),

            # Add the calculated value to the chart
            go.Bar(
                x = ['Your Calculation'],
                y = [rounded_total_co2_emissions],
                name = 'Your Calculation',
                marker = dict(color='#3288bd')
            )
        ]


        # step 4: Generate the Plotly chart
        # Electricity chart
        electricity_layout = go.Layout(
            title = {'text':'Electricity Production vs Benchmarks Comparison', 'x':0.5, 'xanchor':'center'},
            xaxis = dict(title='Benchmark - Year'),
            yaxis = dict(title='Total Electricity Requirement (TWh)'),
            showlegend=False
        )

        electricity_fig = go.Figure(data=electricity_chart_data, layout=electricity_layout)

        # CO2 chart
        co2_layout = go.Layout(
            title = {'text':'CO2 Emissions vs Benchmarks Comparison', 'x':0.5, 'xanchor':'center'},
            xaxis = dict(title='Benchmark - Year'),
            yaxis = dict(title='Total CO2 Emissions (MtCO2)'),
            showlegend=False
        )
        
        co2_fig = go.Figure(data=co2_chart_data, layout=co2_layout)
        

        # step 5: Convert the Plotly chart to HTML
        electricity_chart_html = plot(electricity_fig, output_type='div', include_plotlyjs=False)
        co2_chart_html = plot(co2_fig, output_type='div', include_plotlyjs=False)
    
        # Pass results to the template
        # Construct the context with form and results
        # context = {
        #     'total_electricity_requirement': rounded_total_electricity_requirement,
        #     'total_co2_emissions': rounded_total_co2_emissions,
        #     'co2_emissions_reduction': co2_emissions_reduction,
        #     'total_h2_production': rounded_total_h2_production,
        #     'required_electrolyzer_units': rounded_required_electrolyzer_units,
        #     'electricity_chart_html': electricity_chart_html,
        #     'co2_chart_html': co2_chart_html,
        # }

        print(f'total_electricity_requirement: {rounded_total_electricity_requirement}')
        print(f'total_co2_emissions: {rounded_total_co2_emissions}')
        print(f'co2_emissions_reduction: {co2_emissions_reduction}')
        print(f'total_h2_production: {rounded_total_h2_production}')

        return JsonResponse({
            'total_electricity_requirement': rounded_total_electricity_requirement,
            'total_co2_emissions': rounded_total_co2_emissions,
            'co2_emissions_reduction': co2_emissions_reduction,
            'total_h2_production': rounded_total_h2_production,
            'required_electrolyzer_units': rounded_required_electrolyzer_units,
            'electricity_chart_html': electricity_chart_html,
            'co2_chart_html': co2_chart_html,
        })



        # Render the template with context
        # return render(request, 'h2_prod/hydrogen_production.html', context)
    
    return render(request, 'h2_prod/hydrogen_production.html', context)


'initial_h2_production': 90,  # MegaTonnes
'initial_total_electricity_requirement': 4281,  # TWh
'initial_required_electrolyzer_units': 24437,  # Units
'initial_total_co2_emissions': 1409  # MegaTonnes